In [1]:
import sys, time, pickle
import torch

sys.path.insert(0, '../../Utils')
from global_constants import gpu_device

sys.path.insert(0, '../')
from BERT_models import BERT_QNLI_MODEL

from _utils import sample_random_glue_qnli, get_continuation_mapping, \
                    get_continuous_attributions, get_continuous_raw_inputs, \
                    collect_info_for_metric, save_info, download_HTML

In [2]:
qnli_data_raw, targets, idxs = sample_random_glue_qnli()

Reusing dataset glue (/home/user/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-e6d0a9c9270b6822.arrow
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-c46c5e78d2bda808.arrow


In [3]:
model = BERT_QNLI_MODEL()
tokenizer = model.tokenizer

In [4]:
#define some containers to save some info
model_out_list, raw_attr_list, conti_attr_list, raw_input_list = [], [], [], []

In [5]:
from captum.attr import IntegratedGradients
from captum.attr import visualization 

In [6]:
IG = IntegratedGradients(model)

In [7]:
def generate_record(raw_datum, target): #raw_datum expected to be a tuple/list of 2 sentences
    #tokenizer operations
    tokenized = tokenizer(raw_datum, truncation=True, return_offsets_mapping=True)
    offset_mappings = tokenized['offset_mapping']
    #concatenate the two offset_mappings together because they are fed in together
    conti_map = get_continuation_mapping(offset_mappings[0]) + get_continuation_mapping(offset_mappings[1])
    #change the first input_id of the second sentence to be the last input_id of the 1st sentence (i.e. an [END] token))
    tokenized_input_ids = tokenized['input_ids'][0] + \
                        [tokenized['input_ids'][1][i] if i != 0 else tokenized['input_ids'][0][-1] \
                         for i in range(len(tokenized['input_ids'][1]))]
    input_ids = torch.tensor(tokenized_input_ids).unsqueeze(0).to(gpu_device)
    detokenized = [t.replace('#', '') for t in tokenizer.convert_ids_to_tokens(input_ids[0])]
    
    #feeding input forward 
    input_emb = model.get_embeddings(input_ids)
    pred_prob = model(input_emb).item()

     #categorizing results
    pred_class = 'No Entailment' if pred_prob < 0.5 else 'Entailment' 
    true_class = 'No Entailment' if target < 0.5 else 'Entailment' 
    
    #attribution algorithm working
    attribution, delta = IG.attribute(input_emb, return_convergence_delta=True)
    word_attributions = attribution.squeeze(0).sum(dim=1)
#     word_attributions = attr_normalizing_func(word_attributions)
    word_attributions /= torch.norm(word_attributions)
    attr_score = torch.sum(word_attributions)
    attr_class = 'No Entailment' if attr_score < 0.5 else 'Entailment'
    convergence_score = delta
    
    
#     #re-organizing tensors and arrays because words get split down
    conti_attr = get_continuous_attributions(conti_map, word_attributions)
    raw_input = get_continuous_raw_inputs(conti_map, detokenized)

#     print(f'word attributions {word_attributions}')
    print(f'pred_prob {pred_prob}')
#     print(f'pred_class {pred_class}')
#     print(f'true_class {true_class}')
#     print(f'attribution {attribution}')
#     print(f'attr_class {attr_class}')
#     print(f'attr_score {attr_score}')
#     print(f'raw_input {raw_input}')

        
# #     collect info for metrics later
    collect_info_for_metric(model_out_list, pred_prob, raw_attr_list, attribution, conti_attr_list, conti_attr, raw_input_list, raw_input)
        
    
    visual_record = visualization.VisualizationDataRecord(word_attributions=conti_attr,
                                                         pred_prob=pred_prob,
                                                         pred_class=pred_class,
                                                         true_class=true_class,
                                                         attr_class=attr_class,
                                                         attr_score=attr_score,
                                                         raw_input_ids=raw_input,
                                                         convergence_score=convergence_score)
        
        
    return visual_record
      
    

In [8]:
for i, (datum_raw, target) in enumerate(zip(qnli_data_raw, targets), start=1):
#     example_1 = "How many times has the South Florida/Miami area hosted the Super Bowl?"
#     example_2 = "The South Florida/Miami area has previously hosted the event 10 times (tied for most with New Orleans), with the most recent one being Super Bowl XLIV in 2010."
    
#     example_1 = "When did the third Digimon series begin?"
#     example_2 = "Unlike the two seasons before it and most of the seasons that followed, Digimon Tamers takes a darker and more realistic approach to its story featuring Digimon who do not reincarnate after their deaths and more complex character development in the original Japanese."
#     datum_raw, target = [example_1, example_2], 1
    print(f'Raw datum: {datum_raw}') #datum expected to be a list of 2 sentences
    print(f'GT target: {target}')
    visual_record=generate_record(datum_raw, target)
    print(visualization.visualize_text([visual_record])) 


Raw datum: ['What would a teacher assess the levels of a student on?', 'For example, an experienced teacher and parent described the place of a teacher in learning as follows: "The real bulk of learning takes place in self-study and problem solving with a lot of feedback around that loop.']
GT target: 0
word attr tensor([ 0.2840,  0.0259, -0.0395,  0.0779, -0.0405, -0.0070,  0.0611, -0.1699,
         0.0087,  0.0431, -0.0611, -0.0646, -0.0579,  0.5147,  0.3289, -0.0270,
         0.0077, -0.0770, -0.0241, -0.1167, -0.1098, -0.0618, -0.0351,  0.0627,
         0.0109, -0.1920, -0.0073,  0.0216, -0.0535,  0.0160, -0.2031,  0.0157,
        -0.0993,  0.0044,  0.1890,  0.1058, -0.0249, -0.0162, -0.0582, -0.0810,
         0.2040,  0.0276,  0.0214,  0.1012, -0.0306, -0.1019,  0.0800,  0.1731,
         0.1302,  0.0663,  0.0319,  0.0387,  0.0539,  0.0117,  0.1007,  0.1069,
         0.0069,  0.0419, -0.3844], device='cuda:0', dtype=torch.float64,
       grad_fn=<DivBackward0>)
conti attr [tensor(0

<IPython.core.display.HTML object>
Raw datum: ['What company created Doctor Who?', 'Who character by BBC Television in the early 1960s, a myriad of stories have been published about Doctor Who, in different media: apart from the actual television episodes that continue to be produced by the BBC, there have also been novels, comics, short stories, audio books, radio plays, interactive video games, game books, webcasts, DVD extras, and even stage performances.']
GT target: 1
word attr tensor([ 0.0192, -0.0350,  0.0374,  0.1147, -0.4170, -0.0890,  0.3477,  0.4633,
         0.2496,  0.0664, -0.1533, -0.0220,  0.1108, -0.0588, -0.0202, -0.0021,
        -0.0071, -0.0988, -0.0401,  0.0224, -0.0710,  0.0202, -0.0237,  0.0138,
        -0.0187, -0.0729, -0.0927, -0.3317, -0.1865, -0.0901, -0.0379, -0.0204,
         0.0586,  0.0449, -0.0094,  0.0214,  0.0258,  0.0102,  0.0413, -0.0073,
         0.0149, -0.0364,  0.0446,  0.0535,  0.1144,  0.1299,  0.0482,  0.2066,
         0.0211,  0.0858,  0.103

<IPython.core.display.HTML object>
Raw datum: ['What was the name of the Media Day event for Super Bowl 50?', "The game's media day, which was typically held on the Tuesday afternoon prior to the game, was moved to the Monday evening and re-branded as Super Bowl Opening Night."]
GT target: 1
word attr tensor([-0.1667, -0.1750, -0.1299,  0.0269,  0.0601,  0.0701,  0.0734, -0.3130,
        -0.0842, -0.0530,  0.1080,  0.0060,  0.1990, -0.0330, -0.1485,  0.2995,
         0.3755,  0.0283,  0.2545, -0.0048,  0.0042, -0.4663, -0.0588,  0.0370,
        -0.0009,  0.0143, -0.1311,  0.0057,  0.0379,  0.0101, -0.0616, -0.1305,
         0.1308,  0.0333,  0.0149,  0.1633, -0.0426, -0.0676, -0.0988,  0.0502,
         0.0308, -0.0152, -0.0695,  0.0762,  0.0937,  0.0349,  0.1512,  0.1078,
         0.0442,  0.0408,  0.0249,  0.0952,  0.1265,  0.1413], device='cuda:0',
       dtype=torch.float64, grad_fn=<DivBackward0>)
conti attr [tensor(-0.1667, device='cuda:0', dtype=torch.float64,
       grad_fn=<Unb

<IPython.core.display.HTML object>
Raw datum: ['How many Doctor Who soundtracks have been released since 2005?', 'The fourth was released on 4 October 2010 as a two disc special edition and contained music from the 2008–2010 specials (The Next Doctor to End of Time Part 2).']
GT target: 0
word attr tensor([ 0.2470,  0.0160, -0.0582, -0.2237, -0.0535, -0.0021,  0.0200,  0.0925,
         0.0226, -0.0960,  0.1181,  0.1263,  0.3179,  0.0800,  0.0066, -0.5453,
         0.0857,  0.0140,  0.0693,  0.1211,  0.0441,  0.1958,  0.0930,  0.0447,
         0.0316,  0.0126,  0.0469, -0.0032,  0.0383, -0.0880, -0.0533,  0.0194,
         0.0513,  0.0431,  0.1114,  0.0169, -0.0341,  0.0801,  0.0521,  0.0123,
        -0.0848,  0.0770, -0.0138,  0.0017,  0.0136,  0.0817,  0.1124,  0.0527,
         0.0462,  0.5196], device='cuda:0', dtype=torch.float64,
       grad_fn=<DivBackward0>)
conti attr [tensor(0.2470, device='cuda:0', dtype=torch.float64, grad_fn=<UnbindBackward0>), tensor(0.0160, device='cuda:0',

<IPython.core.display.HTML object>
Raw datum: ["What is the name of the country's longest continuously running student film society?", 'Students at the University of Chicago run over 400 clubs and organizations known as Recognized Student Organizations (RSOs).']
GT target: 0
word attr tensor([-0.1352, -0.0044,  0.1199,  0.1279,  0.3796,  0.0690,  0.0608,  0.0192,
        -0.0293,  0.0353,  0.0043, -0.0066,  0.0039, -0.0422, -0.0470, -0.1921,
         0.1815,  0.3194,  0.2170,  0.0763,  0.2299,  0.0833,  0.1006,  0.0680,
         0.1914,  0.3408,  0.1541,  0.2854,  0.1822, -0.0497,  0.1528,  0.0342,
         0.1635,  0.0726, -0.0133,  0.3065,  0.1410, -0.0311,  0.0412,  0.0096,
         0.0320,  0.1570], device='cuda:0', dtype=torch.float64,
       grad_fn=<DivBackward0>)
conti attr [tensor(-0.1352, device='cuda:0', dtype=torch.float64,
       grad_fn=<UnbindBackward0>), tensor(-0.0044, device='cuda:0', dtype=torch.float64,
       grad_fn=<UnbindBackward0>), tensor(0.1199, device='cuda:

...

<IPython.core.display.HTML object>
Raw datum: ['Who did Britain exploit in India?', 'Although a substantial number of colonies had been designed to provide economic profit and to ship resources to home ports in the seventeenth and eighteenth centuries, Fieldhouse suggests that in the nineteenth and twentieth centuries in places such as Africa and Asia, this idea is not necessarily valid:']
GT target: 0
word attr tensor([ 0.1960,  0.1059, -0.0494, -0.0909, -0.1049,  0.0178, -0.1496,  0.0892,
         0.5796,  0.2992,  0.2050, -0.0379,  0.0086, -0.0779, -0.0386, -0.0127,
        -0.0864, -0.0838, -0.1851,  0.0106, -0.0522, -0.0109, -0.0284, -0.0719,
        -0.0655,  0.0411, -0.0056, -0.0240, -0.0439,  0.0098, -0.0382,  0.0169,
        -0.0207,  0.0190,  0.2372,  0.0972, -0.0367, -0.0753,  0.0107,  0.0552,
         0.0435, -0.0379,  0.0124,  0.1060, -0.0008,  0.0501,  0.1192,  0.0808,
         0.0580, -0.0148,  0.0125,  0.0950,  0.0816,  0.1229,  0.0584,  0.0768,
        -0.0453, -0.0107

<IPython.core.display.HTML object>
Raw datum: ['What popular environmentalist is also a university alumni member?', 'In science, alumni include astronomers Carl Sagan, a prominent contributor to the scientific research of extraterrestrial life, and Edwin Hubble, known for "Hubble\'s Law", NASA astronaut John M. Grunsfeld, geneticist James Watson, best known as one of the co-discoverers of the structure of DNA, experimental physicist Luis Alvarez, popular environmentalist David Suzuki, balloonist Jeannette Piccard, biologists Ernest Everett']
GT target: 1
word attr tensor([ 2.3877e-01, -1.6756e-02,  8.9161e-02,  4.5562e-02, -2.1262e-02,
        -9.6301e-03,  1.1707e-03,  1.5272e-02,  1.0234e-03, -2.5577e-02,
        -1.2245e-02, -8.2443e-02,  3.9939e-01,  3.4574e-01, -3.1161e-02,
        -1.5933e-02, -8.1549e-02,  2.3925e-02, -2.1663e-03, -1.7001e-01,
        -7.2143e-02, -2.6815e-02, -3.3958e-02, -6.4889e-02, -8.7949e-03,
         2.4875e-02, -1.4876e-02, -2.8729e-02,  3.7285e-03,  3.5

<IPython.core.display.HTML object>
Raw datum: ['What publication did Philip Howard work for?', 'Responding to the findings of the survey in The Times newspaper, journalist Philip Howard maintained that, "to compare the violence of Dr Who, sired by a horse-laugh out of a nightmare, with the more realistic violence of other television series, where actors who look like human beings bleed paint that looks like blood, is like comparing Monopoly with the property market in London: both are fantasies, but one is meant to be taken seriously."']
GT target: 1
word attr tensor([-1.9666e-01,  5.4284e-02, -7.3028e-02,  3.8330e-02, -2.3856e-01,
        -4.5005e-01, -2.5867e-02,  7.4719e-03,  1.8391e-02,  3.0038e-01,
         1.8569e-01, -1.0384e-01,  3.2095e-02,  8.0662e-03, -9.1042e-03,
        -2.5309e-02, -4.2670e-03, -5.1384e-02,  1.5334e-01, -2.5176e-02,
         5.8094e-03, -5.4397e-02, -3.2025e-03,  3.6207e-03, -2.6859e-01,
        -5.5939e-01, -2.3831e-02,  6.9080e-03,  2.4054e-03,  1.0105e

<IPython.core.display.HTML object>
Raw datum: ['If the apparant force of two fermions is attractive, what is the spin function?', 'If two identical fermions (e.g. electrons) have a symmetric spin function (e.g. parallel spins) the spatial variables must be antisymmetric (i.e. they exclude each other from their places much as if there was a repulsive force), and vice versa, i.e. for antiparallel spins the position variables must be symmetric (i.e. the apparent force must be attractive).']
GT target: 1
word attr tensor([ 1.7350e-01, -4.0283e-02,  1.5873e-02, -2.6940e-02, -3.5854e-02,
         5.2562e-03, -1.6931e-03, -2.7646e-02,  1.8703e-02, -9.2734e-04,
         4.3806e-02,  1.8653e-02,  7.1348e-03, -3.0114e-01, -5.7346e-02,
         3.4843e-02, -1.8558e-02,  3.0540e-02,  7.3845e-02, -2.9883e-01,
         2.0921e-02,  4.1419e-01,  2.4256e-01, -1.6625e-02, -7.9738e-03,
        -1.7185e-02, -6.7436e-03,  1.4287e-02,  9.6808e-03,  2.9435e-02,
        -2.9806e-02, -6.1813e-03,  1.8068e-02,

<IPython.core.display.HTML object>


In [9]:
file_name_base = 'Integrated_Gradients'

In [10]:
save_info(idxs, qnli_data_raw, targets, model_out_list, raw_attr_list, conti_attr_list, raw_input_list, 
          fname=f'../MethodOutputs/{file_name_base}_out.pkl')

{'indices': [4013,
  2281,
  2771,
  5109,
  1852,
  2016,
  642,
  3912,
  4427,
  2330,
  4581,
  2611,
  5186,
  2700,
  4920,
  4335,
  1246,
  2811,
  5290,
  5385,
  4774,
  653,
  2816,
  916,
  3297,
  3459,
  1475,
  1287,
  4548,
  1476,
  2328,
  2208,
  2772,
  5301,
  869,
  4968,
  4456,
  4711,
  1929,
  634,
  3774,
  5389,
  497,
  4901,
  4602,
  831,
  1292,
  1087,
  3159,
  3492],
 'raw_data': [['What would a teacher assess the levels of a student on?',
   'For example, an experienced teacher and parent described the place of a teacher in learning as follows: "The real bulk of learning takes place in self-study and problem solving with a lot of feedback around that loop.'],
  ['What company created Doctor Who?',
   'Who character by BBC Television in the early 1960s, a myriad of stories have been published about Doctor Who, in different media: apart from the actual television episodes that continue to be produced by the BBC, there have also been novels, comics, sho

In [11]:
download_HTML(cur_file_name=f'{file_name_base}_QNLI_BERT.ipynb',
              out_file_name=f'{file_name_base}_QNLI_BERT.html')